<a href="https://colab.research.google.com/github/nahbos/AUT-Advanced-Information-Retrieval/blob/main/Ex02/neural_network_based_information_retrieval.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Sobhan Moradian Daghigh

- 12-13-2022

### Ex-02: NN-based IR

In [1]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.8 MB 10.7 MB/s 
     |████████████████████████████████| 7.6 MB 58.3 MB/s 
     |████████████████████████████████| 182 kB 60.8 MB/s 


In [2]:
import numpy as np
import pandas as pd
import scipy
import re
from gensim.models import TfidfModel
from gensim.corpora import Dictionary
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import remove_stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from bs4 import BeautifulSoup
from collections import Counter
import random
import pickle
from tqdm import tqdm
from PIL import Image


import transformers
import torch
from transformers import AdamW, AutoTokenizer, AutoModelForSequenceClassification, Trainer
from transformers import TrainingArguments
from transformers import DataCollatorWithPadding

In [3]:
!wget -nc https://raw.githubusercontent.com/nahbos/Advanced-Information-Retrieval/main/Ex01/Data/train_data.csv
!wget -nc https://raw.githubusercontent.com/nahbos/Advanced-Information-Retrieval/main/Ex01/Data/valid_data.csv
!wget -nc https://raw.githubusercontent.com/nahbos/Advanced-Information-Retrieval/main/Ex01/Data/test_data.csv
!wget -nc https://raw.githubusercontent.com/nahbos/Advanced-Information-Retrieval/main/Ex01/Data/bigram_LM.png
!wget -nc https://raw.githubusercontent.com/nahbos/Advanced-Information-Retrieval/main/Ex01/Data/diricklet_smoothing_LM.png

--2022-12-13 18:53:00--  https://raw.githubusercontent.com/nahbos/Advanced-Information-Retrieval/main/Ex01/Data/train_data.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5148485 (4.9M) [text/plain]
Saving to: ‘train_data.csv’

train_data.csv      100%[===================>]   4.91M  --.-KB/s    in 0.03s   

2022-12-13 18:53:01 (169 MB/s) - ‘train_data.csv’ saved [5148485/5148485]

--2022-12-13 18:53:01--  https://raw.githubusercontent.com/nahbos/Advanced-Information-Retrieval/main/Ex01/Data/valid_data.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting 

# Part One. 
* Data Loading

In [4]:
train = pd.read_csv('./train_data.csv')
val   = pd.read_csv('./valid_data.csv')
test  = pd.read_csv('./test_data.csv')

In [5]:
train.head()

,id,qid1,qid2,question1,question2,is_duplicate
0,14,29,30,What are the laws to change your status from a...,What are the laws to change your status from a...,0
1,18,37,38,Why are so many Quora users posting questions ...,Why do people ask Quora questions which can be...,1
2,38,77,78,How do we prepare for UPSC?,How do I prepare for civil service?,1
3,58,117,118,I was suddenly logged off Gmail. I can't remem...,I can't remember my Gmail password or my recov...,1
4,60,121,122,How do I download content from a kickass torre...,Is Kickass Torrents trustworthy?,0


In [6]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37250 entries, 0 to 37249
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   id            37250 non-null  int64 
 1   qid1          37250 non-null  int64 
 2   qid2          37250 non-null  int64 
 3   question1     37250 non-null  object
 4   question2     37250 non-null  object
 5   is_duplicate  37250 non-null  int64 
dtypes: int64(4), object(2)
memory usage: 1.7+ MB


In [7]:
val.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1010 entries, 0 to 1009
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   id            1010 non-null   int64 
 1   qid1          1010 non-null   int64 
 2   qid2          1010 non-null   int64 
 3   question1     1010 non-null   object
 4   question2     1010 non-null   object
 5   is_duplicate  1010 non-null   int64 
dtypes: int64(4), object(2)
memory usage: 47.5+ KB


In [8]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 980 entries, 0 to 979
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   id            980 non-null    int64 
 1   qid1          980 non-null    int64 
 2   qid2          980 non-null    int64 
 3   question1     980 non-null    object
 4   question2     980 non-null    object
 5   is_duplicate  980 non-null    int64 
dtypes: int64(4), object(2)
memory usage: 46.1+ KB


## Preprocess

In [9]:
def preprocess(q):
    
    q = str(q).lower().strip()
    
    # Replace certain special characters with their string equivalents
    q = q.replace('%', ' percent')
    q = q.replace('$', ' dollar ')
    q = q.replace('₹', ' rupee ')
    q = q.replace('€', ' euro ')
    q = q.replace('@', ' at ')
    
    # The pattern '[math]' appears around 900 times in the whole dataset.
    q = q.replace('[math]', '')
    
    # Replacing some numbers with string equivalents (not perfect, can be done better to account for more cases)
    q = q.replace(',000,000,000 ', 'b ')
    q = q.replace(',000,000 ', 'm ')
    q = q.replace(',000 ', 'k ')
    q = re.sub(r'([0-9]+)000000000', r'\1b', q)
    q = re.sub(r'([0-9]+)000000', r'\1m', q)
    q = re.sub(r'([0-9]+)000', r'\1k', q)
    
    # Decontracting words
    # https://en.wikipedia.org/wiki/Wikipedia%3aList_of_English_contractions
    # https://stackoverflow.com/a/19794953

    contractions = { 
    "ain't": "am not",
    "aren't": "are not",
    "can't": "can not",
    "can't've": "can not have",
    "'cause": "because",
    "could've": "could have",
    "couldn't": "could not",
    "couldn't've": "could not have",
    "didn't": "did not",
    "doesn't": "does not",
    "don't": "do not",
    "hadn't": "had not",
    "hadn't've": "had not have",
    "hasn't": "has not",
    "haven't": "have not",
    "he'd": "he would",
    "he'd've": "he would have",
    "he'll": "he will",
    "he'll've": "he will have",
    "he's": "he is",
    "how'd": "how did",
    "how'd'y": "how do you",
    "how'll": "how will",
    "how's": "how is",
    "i'd": "i would",
    "i'd've": "i would have",
    "i'll": "i will",
    "i'll've": "i will have",
    "i'm": "i am",
    "i've": "i have",
    "isn't": "is not",
    "it'd": "it would",
    "it'd've": "it would have",
    "it'll": "it will",
    "it'll've": "it will have",
    "it's": "it is",
    "let's": "let us",
    "ma'am": "madam",
    "mayn't": "may not",
    "might've": "might have",
    "mightn't": "might not",
    "mightn't've": "might not have",
    "must've": "must have",
    "mustn't": "must not",
    "mustn't've": "must not have",
    "needn't": "need not",
    "needn't've": "need not have",
    "o'clock": "of the clock",
    "oughtn't": "ought not",
    "oughtn't've": "ought not have",
    "shan't": "shall not",
    "sha'n't": "shall not",
    "shan't've": "shall not have",
    "she'd": "she would",
    "she'd've": "she would have",
    "she'll": "she will",
    "she'll've": "she will have",
    "she's": "she is",
    "should've": "should have",
    "shouldn't": "should not",
    "shouldn't've": "should not have",
    "so've": "so have",
    "so's": "so as",
    "that'd": "that would",
    "that'd've": "that would have",
    "that's": "that is",
    "there'd": "there would",
    "there'd've": "there would have",
    "there's": "there is",
    "they'd": "they would",
    "they'd've": "they would have",
    "they'll": "they will",
    "they'll've": "they will have",
    "they're": "they are",
    "they've": "they have",
    "to've": "to have",
    "wasn't": "was not",
    "we'd": "we would",
    "we'd've": "we would have",
    "we'll": "we will",
    "we'll've": "we will have",
    "we're": "we are",
    "we've": "we have",
    "weren't": "were not",
    "what'll": "what will",
    "what'll've": "what will have",
    "what're": "what are",
    "what's": "what is",
    "what've": "what have",
    "when's": "when is",
    "when've": "when have",
    "where'd": "where did",
    "where's": "where is",
    "where've": "where have",
    "who'll": "who will",
    "who'll've": "who will have",
    "who's": "who is",
    "who've": "who have",
    "why's": "why is",
    "why've": "why have",
    "will've": "will have",
    "won't": "will not",
    "won't've": "will not have",
    "would've": "would have",
    "wouldn't": "would not",
    "wouldn't've": "would not have",
    "y'all": "you all",
    "y'all'd": "you all would",
    "y'all'd've": "you all would have",
    "y'all're": "you all are",
    "y'all've": "you all have",
    "you'd": "you would",
    "you'd've": "you would have",
    "you'll": "you will",
    "you'll've": "you will have",
    "you're": "you are",
    "you've": "you have"
    }

    q_decontracted = []

    for word in q.split():
        if word in contractions:
            word = contractions[word]

        q_decontracted.append(word)

    q = ' '.join(q_decontracted)
    q = q.replace("'ve", " have")
    q = q.replace("n't", " not")
    q = q.replace("'re", " are")
    q = q.replace("'ll", " will")
    
    # Removing HTML tags
    q = BeautifulSoup(q)
    q = q.get_text()
    
    # Remove punctuations
    pattern = re.compile('\W')
    q = re.sub(pattern, ' ', q).strip()

    
    return q

In [10]:
train_org = train.copy()
train['question1'] = train['question1'].apply(preprocess)
train['question2'] = train['question2'].apply(preprocess)

val_org = val.copy()
val['question1'] = val['question1'].apply(preprocess)
val['question2'] = val['question2'].apply(preprocess)

test_org = test.copy()
test['question1'] = test['question1'].apply(preprocess)
test['question2'] = test['question2'].apply(preprocess)

**Ok, Everything looks right ))**

# Part Two.
Bert

In [11]:
checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [70]:
def tokenize_function(dataset, padding=False, return_tensors=None):
    temp1 = []
    temp2 = []
    labels = []
    for data in tqdm(list(dataset.iterrows())):
        question1 = data[1]['question1']
        question2 = data[1]['question2']
        label     = data[1]['is_duplicate']

        temp1.append(question1)
        temp2.append(question2)
        labels.append(label)
  
    tokens = tokenizer(temp1, temp2, truncation=True) if (padding==False and return_tensors==None) else tokenizer(temp1, temp2, truncation=True, padding=True, return_tensors='pt')
    return tokens, labels

In [66]:
tokenized_train, labels_train = tokenize_function(train)
tokenized_val, labels_val = tokenize_function(val)
tokenized_test, labels_test = tokenize_function(test, padding=True, return_tensors='pt')

100%|██████████| 980/980 [00:00<00:00, 89674.67it/s]


In [14]:
# Create torch dataset
class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels=None):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        if self.labels:
            item["labels"] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.encodings["input_ids"])

In [67]:
tokenized_train = Dataset(tokenized_train, labels_train)
tokenized_val = Dataset(tokenized_val, labels_val)

In [16]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [17]:
training_args = TrainingArguments("./quora-saved-model", evaluation_strategy="epoch", save_strategy='no', 
                                  report_to='none', num_train_epochs=3, 
                                  per_device_train_batch_size=16, per_device_eval_batch_size=16)

trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_val,
    data_collator=data_collator,
    tokenizer=tokenizer,
    )

In [18]:
trainer.train()

/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 37250
  Num Epochs = 3
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 3495
  Number of trainable parameters = 109483778
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss
1,0.178000,0.111942
2,0.113600,0.101338
3,0.069300,0.093243


***** Running Evaluation *****
  Num examples = 1010
  Batch size = 32
***** Running Evaluation *****
  Num examples = 1010
  Batch size = 32
***** Running Evaluation *****
  Num examples = 1010
  Batch size = 32


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=3495, training_loss=0.1316508477337882, metrics={'train_runtime': 1007.1946, 'train_samples_per_second': 110.952, 'train_steps_per_second': 3.47, 'total_flos': 2957238762273840.0, 'train_loss': 0.1316508477337882, 'epoch': 3.0})

In [55]:
def prediction(tokens):
    logits = model(**tokens).logits
    logits = logits.cpu().detach().numpy()
    preds = np.argmax(logits, axis=-1)
    return preds

In [87]:
for part in range(0, len(test), 200):
    tokenized_test_part, labels_test_part = tokenize_function(test[part:part+200], padding=True, return_tensors='pt')
    tokenized_test_part.to("cuda")
    preds = prediction(tokenized_test_part)

    print(labels_test_part)

100%|██████████| 200/200 [00:00<00:00, 20109.33it/s]


[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


100%|██████████| 200/200 [00:00<00:00, 38375.99it/s]


[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


100%|██████████| 200/200 [00:00<00:00, 50072.27it/s]


[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


100%|██████████| 200/200 [00:00<00:00, 53471.49it/s]


[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


100%|██████████| 180/180 [00:00<00:00, 45706.18it/s]


[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


# Part Four.
* Evaluation Metrics

In [ ]:
groups = test_org.groupby(by='question1')
groups.first()

,id,qid1,qid2,question2,is_duplicate
question1,,,,,
Can Donald Trump really become President of US?,38545,69984,69985,What are the chances that Donald Trump will be...,1
Can I grow height after 19 years?,98278,149375,157481,Is it possible to increase your height after 19?,1
Can I locate someones phone without them knowing or having apps?,51470,91290,52786,How can I locate a cell phone location?,1
Can I sell my business idea?,107485,175643,112223,Where can I sell my idea?,1
Can we expect time travel to become a reality?,21100,39755,1183,Do you think time travel is possible?,1
...,...,...,...,...,...
Why do people still think the the world is flat?,4014,7948,7949,Why do some people today still believe that th...,1
Why do some answers collapse in Quora?,8574,4902,16705,Why are some answers collapsed?,1
Why do we fall in love?,98453,163595,136954,"What makes people ""fall"" in love?",1


In [ ]:
for i, (grp, pdf) in enumerate(groups):
    print(' > {:65} : # of relevant questions: {}'.format(grp, len(pdf)))
    if i == 5: break

 > Can Donald Trump really become President of US?                   : # of relevant questions: 8
 > Can I grow height after 19 years?                                 : # of relevant questions: 1
 > Can I locate someones phone without them knowing or having apps?  : # of relevant questions: 2
 > Can I sell my business idea?                                      : # of relevant questions: 6
 > Can we expect time travel to become a reality?                    : # of relevant questions: 25
 > Can you help me get a lot more followers on Instagram?            : # of relevant questions: 5


In [ ]:
def check_relevancy(groups, query, returned_docs):
    reles = []
    for i, (grp, pdf) in enumerate(groups):
        if grp == query: 
            relevances = [ele for ele in pdf.loc[:, 'question2']]
            for doc in returned_docs:
                if doc in relevances:
                    reles.append(1)
                else: 
                    reles.append(0)
            return reles    # Returns an array like: [0, 1, 1, 0, 0, 1, ...]

### Average Precision (AP)

In [ ]:
def eval_AP(arr):
    relevant = 0
    map = 0
    for i, ele in enumerate(arr):
        if ele == 1:
            relevant += 1
            map += relevant / (i + 1)
    return (map / relevant) if relevant != 0 else 0

### Reciprocal Rank (RR)

In [ ]:
def eval_RR(arr):
    for i, ele in enumerate(arr):
        if ele == 1:
            return 1 / (i + 1)
    return 0

### Precision @ k

In [ ]:
def p_at_k(arr, k=10):
    return sum(arr[:k]) / k

### First algorithm evaluation

In [ ]:
results = get_similar_questions(test, train, similarity)

In [ ]:
groups_test = test_org.groupby(by='question1')
evals = []
for i, ele in enumerate(results):
    temp = check_relevancy(groups_test, test_org.loc[i, 'question1'], ele)

    eval_ap = eval_AP(temp)
    eval_rr = eval_RR(temp)
    eval_p_at_5 = p_at_k(temp, k=5)
    eval_p_at_10 = p_at_k(temp)


    evals.append([eval_ap, eval_rr, eval_p_at_5, eval_p_at_10])


arrs = np.array(evals).reshape(-1, order='F').reshape(4, -1)
print('MAP : {:.3f}'.format(np.average(arrs[0])))
print('MRR : {:.3f}'.format(np.average(arrs[1])))
print('P@5 : {:.3f}'.format(np.average(arrs[2])))
print('P@10: {:.3f}'.format(np.average(arrs[3])))

MAP : 0.685
MRR : 0.788
P@5 : 0.528
P@10: 0.437


# Find parameters
**Its time to search for best parameters value using evaluation metrics on validation dataset**

In [ ]:
groups_val = val_org.groupby(by='question1')

In [ ]:
k = 10

In [ ]:
for mu in [10, 50, 100]:
    print('------------------------------------------')
    print('mu: {}'.format(mu))
    print()
    for i, val_q in enumerate(tqdm(tokenized_qs_val[:50])):
        scores = []
        for train_q in tokenized_qs_train:
            prob = 1
            for token in val_q:
                prob = np.multiply(prob, probability_word_given_doc(word=token, doc=' '.join(train_q), collection=collection, collection_length=collection_length, collection_freq=collection_freq, mu=mu))
            scores.append(prob)

        relevant_results = check_relevancy(groups_val, query=val_org.loc[:, 'question1'].unique()[i], returned_docs=list(train_org.loc[:, 'question2'].unique()[np.argsort(scores)[::-1][:k]]))
        eval_ap = eval_AP(relevant_results)
        eval_rr = eval_RR(relevant_results)
        eval_p_at_5 = p_at_k(relevant_results, k=5)
        eval_p_at_10 = p_at_k(relevant_results)
        evals.append([eval_ap, eval_rr, eval_p_at_5, eval_p_at_10])
        
        if i == 7:
            print('\nValidation Q1: ', val_org.loc[:, 'question1'].unique()[i])
            
            relevant_qs = get_relevants(val_org, q1=val_org.loc[:, 'question1'].unique()[i])
            print('Relevants    : ', )
            [print('             > ', q) for q in relevant_qs]

            print('Train        :')
            [print('             > ', q) for q in list(train_org.loc[:, 'question2'].unique()[np.argsort(scores)[::-1][:k]])]

            print('AP: {:.2f}'.format(eval_ap))
            print()

    print('\n---------------------------------------')

    arrs = np.array(evals).reshape(-1, order='F').reshape(4, -1)
    print('MAP : {:.3f}'.format(np.average(arrs[0])))
    print('MRR : {:.3f}'.format(np.average(arrs[1])))
    print('P@5 : {:.3f}'.format(np.average(arrs[2])))
    print('P@10: {:.3f}'.format(np.average(arrs[3])))

------------------------------------------
mu: 10



 16%|█▌        | 8/50 [00:10<00:50,  1.21s/it]


Validation Q1:  How can I stop myself from watching too much of porn?
Relevants    : 
             >  How can I stop watching porn?
             >  How do I stop addiction to porn?
             >  How can I control myself from masturbating while watching porn?
             >  Catholics: how do I stop watching internet porn?
             >  How do I stop being addicted to porn?
             >  What should be done to avoid watching porn?
             >  How do I stop masturbating/watching porn? I can't/don't masturbate without watching porn.
             >  How shall I stop watching porn?
Train        :
             >  How do I stop masturbating/watching porn? I can't/don't masturbate without watching porn.
             >  How can I stop watching porn?
             >  Why can't I stop watching porn?
             >  Should I stop watching porn?
             >  How shall I stop watching porn?
             >  Catholics: how do I stop watching internet porn?
             >  What should be d

100%|██████████| 50/50 [01:12<00:00,  1.46s/it]



---------------------------------------
MAP : 0.676
MRR : 0.777
P@5 : 0.521
P@10: 0.429
------------------------------------------
mu: 50



...


---------------------------------------
MAP : 0.668
MRR : 0.768
P@5 : 0.514
P@10: 0.423
------------------------------------------
mu: 100



 16%|█▌        | 8/50 [00:12<01:00,  1.43s/it]


Validation Q1:  How can I stop myself from watching too much of porn?
Relevants    : 
             >  How can I stop watching porn?
             >  How do I stop addiction to porn?
             >  How can I control myself from masturbating while watching porn?
             >  Catholics: how do I stop watching internet porn?
             >  How do I stop being addicted to porn?
             >  What should be done to avoid watching porn?
             >  How do I stop masturbating/watching porn? I can't/don't masturbate without watching porn.
             >  How shall I stop watching porn?
Train        :
             >  How do I stop masturbating/watching porn? I can't/don't masturbate without watching porn.
             >  How can I stop watching porn?
             >  Why can't I stop watching porn?
             >  Should I stop watching porn?
             >  How shall I stop watching porn?
             >  Catholics: how do I stop watching internet porn?
             >  What should be d

100%|██████████| 50/50 [01:11<00:00,  1.44s/it]


---------------------------------------
MAP : 0.658
MRR : 0.755
P@5 : 0.507
P@10: 0.417


In [ ]:
for mu in [10, 50, 100]:
    for _lambda in [0.1, 0.25]:
        print('------------------------------------------')
        print('mu: {}  -  lambda: {}'.format(mu, _lambda))
        print()
        evals = []
        for i, val_q in enumerate(tqdm(tokenized_qs_val[:30])):
            scores = []
            for train_q in tokenized_qs_train:
                prob = probability_word_given_doc(word=val_q[0], doc=' '.join(train_q), collection=collection, collection_length=collection_length, collection_freq=collection_freq, mu=mu) # P(q1|D)
                last_token = val_q[0]
                for token in val_q[1:]:
                    temp = probability_bigram_word_given_doc(curr_word=token, last_word=last_token, doc=' '.join(train_q), collection=collection, collection_length=collection_length, collection_freq=collection_freq, _lambda=_lambda)
                    prob = np.multiply(prob, temp)
                    last_token = token
                scores.append(prob)

            relevant_results = check_relevancy(groups_val, query=val_org.loc[:, 'question1'].unique()[i], returned_docs=list(train_org.loc[:, 'question2'].unique()[np.argsort(scores)[::-1][:k]]))
            eval_ap = eval_AP(relevant_results)
            eval_rr = eval_RR(relevant_results)
            eval_p_at_5 = p_at_k(relevant_results, k=5)
            eval_p_at_10 = p_at_k(relevant_results)
            evals.append([eval_ap, eval_rr, eval_p_at_5, eval_p_at_10])
            
            if i == 7:
                print('\nValidation Q1: ', val_org.loc[:, 'question1'].unique()[i])
                
                relevant_qs = get_relevants(val_org, q1=val_org.loc[:, 'question1'].unique()[i])
                print('Relevants    : ', )
                [print('             > ', q) for q in relevant_qs]

                print('Train        :')
                [print('             > ', q) for q in list(train_org.loc[:, 'question2'].unique()[np.argsort(scores)[::-1][:k]])]

                print('AP: {:.2f}'.format(eval_ap))
                print()

        print('\n---------------------------------------')

        arrs = np.array(evals).reshape(-1, order='F').reshape(4, -1)
        print('MAP : {:.3f}'.format(np.average(arrs[0])))
        print('MRR : {:.3f}'.format(np.average(arrs[1])))
        print('P@5 : {:.3f}'.format(np.average(arrs[2])))
        print('P@10: {:.3f}'.format(np.average(arrs[3])))

------------------------------------------
mu: 10  -  lambda: 0.1



 27%|██▋       | 8/30 [00:14<00:35,  1.61s/it]


Validation Q1:  How can I stop myself from watching too much of porn?
Relevants    : 
             >  How can I stop watching porn?
             >  How do I stop addiction to porn?
             >  How can I control myself from masturbating while watching porn?
             >  Catholics: how do I stop watching internet porn?
             >  How do I stop being addicted to porn?
             >  What should be done to avoid watching porn?
             >  How do I stop masturbating/watching porn? I can't/don't masturbate without watching porn.
             >  How shall I stop watching porn?
Train        :
             >  How do I stop masturbating/watching porn? I can't/don't masturbate without watching porn.
             >  Should I stop watching porn?
             >  Why can't I stop watching porn?
             >  How can I stop watching porn?
             >  How shall I stop watching porn?
             >  Catholics: how do I stop watching internet porn?
             >  How do I stop be

100%|██████████| 30/30 [00:59<00:00,  2.00s/it]



---------------------------------------
MAP : 0.409
MRR : 0.453
P@5 : 0.320
P@10: 0.253
------------------------------------------
mu: 10  -  lambda: 0.25



...


---------------------------------------
MAP : 0.440
MRR : 0.477
P@5 : 0.327
P@10: 0.250
------------------------------------------
mu: 100  -  lambda: 0.25



 27%|██▋       | 8/30 [00:15<00:35,  1.63s/it]


Validation Q1:  How can I stop myself from watching too much of porn?
Relevants    : 
             >  How can I stop watching porn?
             >  How do I stop addiction to porn?
             >  How can I control myself from masturbating while watching porn?
             >  Catholics: how do I stop watching internet porn?
             >  How do I stop being addicted to porn?
             >  What should be done to avoid watching porn?
             >  How do I stop masturbating/watching porn? I can't/don't masturbate without watching porn.
             >  How shall I stop watching porn?
Train        :
             >  How do I stop masturbating/watching porn? I can't/don't masturbate without watching porn.
             >  How can I stop watching porn?
             >  Why can't I stop watching porn?
             >  Should I stop watching porn?
             >  How shall I stop watching porn?
             >  Catholics: how do I stop watching internet porn?
             >  What should be d

100%|██████████| 30/30 [01:02<00:00,  2.07s/it]


---------------------------------------
MAP : 0.421
MRR : 0.460
P@5 : 0.307
P@10: 0.243


# The parameters found

In [ ]:
groups_test = test_org.groupby(by='question1')

In [ ]:
k = 10 

In [ ]:
mu = 10
for i, test_q in enumerate(tqdm(tokenized_qs_test)):
    scores = []
    for train_q in tokenized_qs_train:
        prob = 1
        for token in test_q:
            prob = np.multiply(prob, probability_word_given_doc(word=token, doc=' '.join(train_q), collection=collection, collection_length=collection_length, collection_freq=collection_freq, mu=mu))
        scores.append(prob)

    relevant_results = check_relevancy(groups_test, query=test_org.loc[:, 'question1'].unique()[i], returned_docs=list(train_org.loc[:, 'question2'].unique()[np.argsort(scores)[::-1][:k]]))
    eval_ap = eval_AP(relevant_results)
    eval_rr = eval_RR(relevant_results)
    eval_p_at_5 = p_at_k(relevant_results, k=5)
    eval_p_at_10 = p_at_k(relevant_results)
    evals.append([eval_ap, eval_rr, eval_p_at_5, eval_p_at_10])
    
    if i == 7:
        print('\nTest Q1: ', test_org.loc[:, 'question1'].unique()[i])
        
        relevant_qs = get_relevants(test_org, q1=test_org.loc[:, 'question1'].unique()[i])
        print('Relevants    : ', )
        [print('             > ', q) for q in relevant_qs]

        print('Train        :')
        [print('             > ', q) for q in list(train_org.loc[:, 'question2'].unique()[np.argsort(scores)[::-1][:k]])]

        print('AP: {:.2f}'.format(eval_ap))
        print()

print('\n---------------------------------------')

arrs = np.array(evals).reshape(-1, order='F').reshape(4, -1)
print('MAP : {:.3f}'.format(np.average(arrs[0])))
print('MRR : {:.3f}'.format(np.average(arrs[1])))
print('P@5 : {:.3f}'.format(np.average(arrs[2])))
print('P@10: {:.3f}'.format(np.average(arrs[3])))

  5%|▌         | 8/146 [00:09<02:56,  1.28s/it]


Test Q1:  Why do people still think the the world is flat?
Relevants    : 
             >  Why do some people today still believe that the Earth is flat? What evidence has convinced them that this is so?
             >  Why do some people think Earth is flat?
             >  Why do some people still believe that the earth is flat?
             >  Why do people still believe the world is flat?
             >  Why do some people currently believe the earth is flat?
             >  How do people still think the Earth is flat even though it's proven wrong?
             >  Why do some people think the Earth is flat?
             >  Why there are people who still believe that earth is flat?
             >  I know some people that STILL believe the earth is flat. Why?
             >  Why do some people believe that the world is flat?
             >  Why do some smart and wise people think Earth is flat?
             >  Why are there still people who think that the Earth is flat?
            

100%|██████████| 146/146 [03:08<00:00,  1.29s/it]


---------------------------------------
MAP : 0.655
MRR : 0.747
P@5 : 0.499
P@10: 0.406


In [ ]:
mu, _lambda = 100, 0.1
evals = []
for i, test_q in enumerate(tqdm(tokenized_qs_test)):
    scores = []
    for train_q in tokenized_qs_train:
        prob = probability_word_given_doc(word=test_q[0], doc=' '.join(train_q), collection=collection, collection_length=collection_length, collection_freq=collection_freq, mu=mu) # P(q1|D)
        last_token = test_q[0]
        for token in test_q[1:]:
            temp = probability_bigram_word_given_doc(curr_word=token, last_word=last_token, doc=' '.join(train_q), collection=collection, collection_length=collection_length, collection_freq=collection_freq, _lambda=_lambda)
            prob = np.multiply(prob, temp)
            last_token = token
        scores.append(prob)

    relevant_results = check_relevancy(groups_test, query=test_org.loc[:, 'question1'].unique()[i], returned_docs=list(train_org.loc[:, 'question2'].unique()[np.argsort(scores)[::-1][:k]]))
    eval_ap = eval_AP(relevant_results)
    eval_rr = eval_RR(relevant_results)
    eval_p_at_5 = p_at_k(relevant_results, k=5)
    eval_p_at_10 = p_at_k(relevant_results)
    evals.append([eval_ap, eval_rr, eval_p_at_5, eval_p_at_10])
    
    if i == 7:
        print('\nTest Q1: ', test_org.loc[:, 'question1'].unique()[i])
        
        relevant_qs = get_relevants(test_org, q1=test_org.loc[:, 'question1'].unique()[i])
        print('Relevants    : ', )
        [print('             > ', q) for q in relevant_qs]

        print('Train        :')
        [print('             > ', q) for q in list(train_org.loc[:, 'question2'].unique()[np.argsort(scores)[::-1][:k]])]

        print('AP: {:.2f}'.format(eval_ap))
        print()

print('\n---------------------------------------')

arrs = np.array(evals).reshape(-1, order='F').reshape(4, -1)
print('MAP : {:.3f}'.format(np.average(arrs[0])))
print('MRR : {:.3f}'.format(np.average(arrs[1])))
print('P@5 : {:.3f}'.format(np.average(arrs[2])))
print('P@10: {:.3f}'.format(np.average(arrs[3])))

  5%|▌         | 8/146 [00:13<03:55,  1.71s/it]


Test Q1:  Why do people still think the the world is flat?
Relevants    : 
             >  Why do some people today still believe that the Earth is flat? What evidence has convinced them that this is so?
             >  Why do some people think Earth is flat?
             >  Why do some people still believe that the earth is flat?
             >  Why do people still believe the world is flat?
             >  Why do some people currently believe the earth is flat?
             >  How do people still think the Earth is flat even though it's proven wrong?
             >  Why do some people think the Earth is flat?
             >  Why there are people who still believe that earth is flat?
             >  I know some people that STILL believe the earth is flat. Why?
             >  Why do some people believe that the world is flat?
             >  Why do some smart and wise people think Earth is flat?
             >  Why are there still people who think that the Earth is flat?
            

100%|██████████| 146/146 [04:10<00:00,  1.72s/it]


---------------------------------------
MAP : 0.507
MRR : 0.538
P@5 : 0.358
P@10: 0.272


# Finito